In [3]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from impyute.imputation.cs import mice, fast_knn
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report, confusion_matrix
from sklearn.svm import SVR

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy

In [4]:
def timestamp_data_treatment(dataset, category='numeric'):
    if category == 'binary':
        for column in ['Mes', 'Hora Medicao']:
            date = dataset[column]
            _max = date.max()
            i = 0
            while 2**i < _max:
                i += 1
            binary_rep = [np.binary_repr(z,width=i) for z in date.astype('int')]
            d = []
            for k in range(i):
                d.append(np.array([int(v) for s in binary_rep for v in s[k]]))
            t = np.vstack((d)).T
            if column == 'Mes':
                df_mes = pd.DataFrame(t, columns= [f'Mes_Bit_{ind}' for ind in range(i)])
            else:
                df_hora = pd.DataFrame(t, columns= [f'Hora_Bit_{ind}' for ind in range(i)])

    elif category == '1ofN':
        for column in ['Mes','Hora Medicao']:
            one_hot = pd.get_dummies(dataset[column])
            listoflists = one_hot.values.tolist()
            new_list = []
            for lists in listoflists:
                new_list.append(''.join(map(str, lists)))
            d = []
            if column == 'Mes':
                for k in range(12):
                        d.append(np.array([int(v) for s in new_list for v in s[k]]))
                t = np.vstack((d)).T
                df_mes = pd.DataFrame(t, columns= ['Jan', 'Fev', 'Mar','Abr','Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez'])
            if column == 'Hora Medicao':
                for k in range(24):
                        d.append(np.array([int(v) for s in new_list for v in s[k]]))
                t = np.vstack((d)).T
                df_hora = pd.DataFrame(t, columns= [f'{ind} h' for ind in range(24)])
            if column == 'Dia':
                for k in range(24):
                        d.append(np.array([int(v) for s in new_list for v in s[k]]))
                t = np.vstack((d)).T
                df_hora = pd.DataFrame(t, columns= [f'{ind} h' for ind in range(24)])
    
    elif category == 'numeric':
        for column in ['Dia', 'Mes','Hora Medicao']:
            _values = dataset[column].values
            _max = dataset[column].max()
            
            _array = np.array([float(i/_max) for i in _values])
            if column == 'Mes':
                df_mes = pd.DataFrame(_array, columns= ['Mes'])
            else:
                df_hora = pd.DataFrame(_array, columns= ['Horas'])
    else:
        print('Invalid Category, please select another one...')

    return df_mes, df_hora    


def preparing_data(dataset, features = [], lag = 24, normalize = 'minmax', category = 'binary', onlyHour = False, onlyMonth = False):
    _max = dataset.iloc[:,-1:].max().values
    _min = dataset.iloc[:,-1:].min().values
    _med = dataset.iloc[:,-1:].mean().values
    _std = dataset.iloc[:,-1:].std().values
    if normalize == 'minmax':
        scaler = MinMaxScaler(feature_range=(0, 1))
        dataset_norm = scaler.fit_transform(dataset.drop([i for i in dataset.columns if i in ['Dia', 'Mes', 'Ano', 'Hora Medicao']],axis=1))
        dataset_norm = pd.DataFrame(dataset_norm, 
                                   columns = dataset.drop([i for i in dataset.columns if i in ['Dia', 'Mes', 'Ano', 'Hora Medicao']],axis=1).columns)
    elif normalize == 'standard':
        scaler = StandardScaler()
        dataset_norm = scaler.fit_transform(dataset.drop([i for i in dataset.columns if i in ['Dia', 'Mes', 'Ano', 'Hora Medicao']],axis=1))
        dataset_norm = pd.DataFrame(dataset_norm, 
                                    columns = dataset.drop([i for i in dataset.columns if i in ['Dia', 'Mes', 'Ano', 'Hora Medicao']],axis=1).columns)
    else:
        print('Invalid Category, please select another one...')
     
    X, Y = look_back_function(dataset_norm['VENTO, VELOCIDADE HORARIA(m/s)'],lag)
    trend = pd.DataFrame(X.iloc[:, -1] - X.iloc[:, -2], columns = ['Trend'])
    X = pd.concat([trend, X], axis = 1)
    mes, hora = timestamp_data_treatment(dataset, category)
    
    if len(features):
        for feature in features:
            if feature in dataset_norm.drop([i for i in dataset_norm.columns if i in ['Dia', 'Mes', 'Ano', 'Hora Medicao']], axis=1).columns:
                X = pd.concat([dataset_norm[feature][lag:-1].reset_index(drop=True), X], axis=1)
        
    if onlyMonth:
        _input = pd.concat([mes[lag:-1].reset_index(drop=True), X], axis=1)
        
    elif onlyHour:
        _input = pd.concat([hora[lag:-1].reset_index(drop=True), X], axis=1)
        
    else:
        _input = pd.concat([mes[lag:-1].reset_index(drop=True), hora[lag:-1].reset_index(drop=True), X], axis=1)
        
    return _input, Y,_max, _min, _med, _std
        

def look_back_function (dataset, look_back=24):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    X = pd.DataFrame(np.array(dataX), columns = [f'X(t-{ind})' for ind in reversed(range(look_back))])
    Y = pd.DataFrame(np.array(dataY), columns=['Output'])
    return X, Y

def multistep_prediction(model, Input, lag):
    other_features = Input.iloc[:, :(Input.shape[1]-lag)]
    other_features_array = other_features.values

    wind_feature = Input.iloc[:,-lag:]
    wind_feature_array = wind_feature.values

    for i in range(len(wind_feature_array)):
        if i==0:
            x_multistep = np.concatenate((other_features_array[i],wind_feature_array[i]))
            _y_hat = model.predict(x_multistep.reshape(1,-1))
            predict_array = np.array(_y_hat)
        else:
            if i < lag:
                wind_array_copy =  wind_feature_array[i, :]
                wind_array_copy = wind_array_copy[:-predict_array.shape[0]]
                data_array_with_prediction = np.insert(wind_array_copy,wind_array_copy.size,predict_array)
            else:
                data_array_with_prediction = predict_array[-lag:]
            x_multistep = np.concatenate((other_features_array[i],data_array_with_prediction))
            _y_hat = model.predict(x_multistep.reshape(1,-1))
            predict_array = np.append(predict_array, _y_hat)
    predict_data = pd.DataFrame(predict_array, columns=['Predictions'])
    return predict_data

def mask_classification(raw_data):
    classification_list = np.array([])
    for idx, x in raw_data.iterrows():
        if 0 <= x.values <= 4:
            classification_list = np.append(classification_list, 1)
        elif 4 < x.values <= 10:
            classification_list = np.append(classification_list, 2)
        elif 10 < x.values <= 15:
            classification_list = np.append(classification_list, 3)
        elif 15 < x.values <= 25:
            classification_list = np.append(classification_list, 4)
        else:
            classification_list = np.append(classification_list, 5)
    return classification_list

def MAPE(testY, testPredict):
    soma, cont = 0, 0
    for idx, value in testY.iterrows():
        if abs(value.values) >= 0.01:
            erro = abs((value - testPredict.values[idx])/value)
            soma += erro
            cont += 1
    mape = (soma/cont)*100
    return mape

In [5]:
df_train_micetreated = pd.read_csv('Treino_Arraial_do_Cabo_MICE.csv', sep = ';')
df_test_micetreated = pd.read_csv('Teste_Arraial_do_Cabo_MICE.csv', sep = ';')

In [ ]:
df = pd.concat([df_train_micetreated, df_test_micetreated])
df = df.drop([i for i in df.columns if i in 
              ['Dia', 'Mes', 'Ano', 'Hora Medicao', 'TEMPERATURA DA CPU DA ESTACAO(°C)', 'TENSAO DA BATERIA DA ESTACAO(V)']],axis=1)
scaler = MinMaxScaler(feature_range=(0, 1))
df_norm = scaler.fit_transform(df)
df_norm = pd.DataFrame(df_norm, columns = df.columns)

**Seleção de Features**

In [ ]:
#Backward Elimination
_x = df_norm.drop([i for i in df_norm.columns if i in ['VENTO, VELOCIDADE HORARIA(m/s)']],axis=1)
_y = df_norm["VENTO, VELOCIDADE HORARIA(m/s)"]
cols = list(_x.columns)
pmax = 1
while (len(cols)>0):
    p= []
    X_1 = _x[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(_y,X_1).fit()
    p = pd.Series(model.pvalues.values[1:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)

In [ ]:
#RFE
nof_list=np.arange(1,df_norm.shape[1])            
high_score=0

nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(_x,_y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))
cols = list(_x.columns)
model = LinearRegression()

#Initializing RFE model
rfe = RFE(model, nof)

#Transforming data using RFE
X_rfe = rfe.fit_transform(_x,_y)

#Fitting the data to model
model.fit(X_rfe,_y)              
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index
print(selected_features_rfe)

In [ ]:
#EMbedded Method
reg = LassoCV()
reg.fit(_x, _y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(_x, _y))
coef = pd.Series(reg.coef_, index = _x.columns)

list_embedded = []
for ind, value in coef.iteritems():
    if value != 0:
        list_embedded.append(ind)
print(list_embedded)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()

mpl.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

In [ ]:
forest = RandomForestRegressor(max_depth = 5, random_state = 42)

boruta_feature_selector = BorutaPy(forest, n_estimators='auto', verbose=0, random_state=4242, max_iter = 100, perc = 75)
boruta_feature_selector.fit(np.array(_x), np.array(_y))

final_features = list()
indexes = np.where(boruta_feature_selector.support_ == True)
for x in np.nditer(indexes):
    final_features.append(df.columns[x])

print(final_features)

**Interseção das Features**

In [ ]:
backward_elimination_list = set(selected_features_BE)
RFE_list = set(selected_features_rfe)
embedded_method_list = set(list_embedded)
boruta_method_list = set(final_features)

BE_RFE_List = backward_elimination_list.intersection(RFE_list)
EM_BE_RFE_List = BE_RFE_List.intersection(embedded_method_list)
final_feature_list = list(EM_BE_RFE_List.intersection(boruta_method_list))

print(final_feature_list)

**Previsão Horizonte de 1 dia**

**Seleção de hiper-parâmetros com algoritmos genéticos**

In [ ]:
from deap import base, creator, tools, algorithms
import random

def mutate(individual):
    
    gene = random.randint(0,7) #select which parameter to mutate
    if gene == 0:
        if individual[0] == 3:
            individual[0] = random.choice([6, 12, 24])
        elif individual[0] == 6:
            individual[0] = random.choice([3, 12, 24])
        elif individual[0] == 12:
            individual[0] = random.choice([3, 6, 24])
        else:
            individual[0] = random.choice([3, 6, 12])
    elif gene == 1:
        if individual[1] == 'minmax':
            individual[1] = 'standard'
        else:
            individual[1] = 'minmax'
    elif gene == 2:
        if individual[2] == 'numeric':
            individual[2] = random.choice(['1ofN', 'binary'])
        elif individual[2] == '1ofN':
            individual[2] = random.choice(['numeric', 'binary'])
        else:
            individual[2] = random.choice(['numeric', '1ofN'])
        
    elif gene == 3:
        if individual[3] == True:
            individual[3] = False
        else:
            individual[3] = True

    elif gene == 4:
        if individual[4] == True:
            individual[4] = False
        else:
            individual[4] = True
    if gene == 5:
        if individual[5] == 'linear':
            individual[5] = random.choice(['poly', 'rbf', 'sigmoid'])
        elif individual[5] == 'poly':
            individual[5] = random.choice(['linear', 'rbf', 'sigmoid'])
        elif individual[5] == 'rbf':
            individual[5] = random.choice(['linear', 'poly', 'sigmoid'])
        else:
            individual[5] = random.choice(['linear', 'poly', 'rbf'])
        
    elif gene == 6:
        individual[6] = random.uniform(lower_C, upper_C)
            
    elif gene == 7:
        individual[7] = random.uniform(lower_epsilon, upper_epsilon)
        
    return individual,


def evaluate(individual):
    '''
    build and test a model based on the parameters in an individual and return
    the AUROC value
    '''
    # extract the values of the parameters from the individual chromosome
    _lag = individual[0]
    _normalize = individual[1]
    _category = individual[2]
    _month = individual[3]
    _hour = individual[4]
    _kernel = individual[5]
    _C = individual[6]
    _epsilon = individual[7]
    
    # build the model
    print(f'Train_Lag: {_lag}')
    print(f'Train_Norm: {_normalize}')
    print(f'Train_Category: {_category}')
    print(f'Train_Hour: {_hour}')
    print(f'Train_Month: {_month}')
    X_train, Y_train,_maxtrain, _mintrain, _meantrain, _stdtrain = preparing_data(df_train_micetreated, features = final_feature_list, lag = _lag, normalize = _normalize, category = _category, onlyHour = _hour, onlyMonth = _month)
    print(f'Test_Lag: {_lag}')
    print(f'Test_Norm: {_normalize}')
    print(f'Test_Category: {_category}')
    print(f'Test_Hour: {_hour}')
    print(f'Test_Month: {_month}')
    X_test, Y_test,_maxtest, _mintest, _meantest, _stdtest = preparing_data(df_test_micetreated, features = final_feature_list,lag = _lag, normalize = _normalize, category = _category, onlyHour = _hour, onlyMonth = _month)
    
    
    print(f'kernel: {_kernel}')
    print(f'C: {_C}')
    print(f'epsilon: {_epsilon}')
    
    regressor = SVR(kernel=_kernel, 
                    C=_C, 
                    epsilon=_epsilon)
    regressor.fit(X_train, Y_train)
    
    testpredictSVR = multistep_prediction(regressor, X_test[-24:], _lag)

    orig_y_eval_test = Y_test[-24:]*(_maxtest - _mintest) + _mintest
    orig_y_hat_test_svr = testpredictSVR*(_maxtest - _mintest) + _mintest

    RMSE_test_svr = (mean_squared_error(orig_y_eval_test, orig_y_hat_test_svr))**0.5
    

    print(f'RMSE: {RMSE_test_svr}')
    print('------'*20)
    return RMSE_test_svr,


creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # Maximise the fitness function value
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Possible parameter values
lag_option = [3, 6, 12, 24]
norm = ['minmax', 'standard']
cat = ['numeric', '1ofN', 'binary']
hour = [True, False]
month = [True, False]
list_kernel = ['linear', 'poly', 'rbf', 'sigmoid']
lower_C, upper_C = 0.1, 50
lower_epsilon, upper_epsilon = 0.001,1 

N_CYCLES = 1
toolbox.register("attr_lag", random.choice, lag_option)
toolbox.register("attr_normalization", random.choice, norm)
toolbox.register("attr_category", random.choice, cat)
toolbox.register("attr_hour", random.choice, hour)
toolbox.register("attr_month", random.choice, month)
toolbox.register("attr_kernel", random.choice, list_kernel)
toolbox.register("attr_C", random.uniform, lower_C, upper_C)
toolbox.register("attr_epochs", random.uniform, lower_epsilon, upper_epsilon)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_lag,toolbox.attr_normalization, toolbox.attr_category,
                  toolbox.attr_hour,toolbox.attr_month,toolbox.attr_kernel,toolbox.attr_C, 
                  toolbox.attr_epochs), n=N_CYCLES)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate",mutate)
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("evaluate", evaluate)

population_size = 4
crossover_probability = 0.75
mutation_probability = 0.05
number_of_generations = 50

pop = toolbox.population(n=population_size)
hof = tools.HallOfFame(3)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=crossover_probability, stats = stats, 
                               mutpb = mutation_probability, ngen=number_of_generations, halloffame=hof, 
                               verbose=True) 

svr_best_parameters_1 = hof[0] # save the optimal set of parameters
svr_best_parameters_2 = hof[1]
svr_best_parameters_3 = hof[2]
print(svr_best_parameters_1)
print(svr_best_parameters_2)
print(svr_best_parameters_3)

gen = log.select("gen")
max_ = log.select("max")
avg = log.select("avg")
min_ = log.select("min")

c:\users\diego\appdata\local\programs\python\python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\users\diego\appdata\local\programs\python\python39\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Train_Lag: 12
Train_Norm: minmax
Train_Category: 1ofN
Train_Hour: True
Train_Month: False
Test_Lag: 12
Test_Norm: minmax
Test_Category: 1ofN
Test_Hour: True
Test_Month: False
kernel: rbf
C: 28.945503351078816
epsilon: 0.0026106660319077115


c:\users\diego\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [ ]:
evolution = pd.DataFrame({'Generation': gen,
                         'Max RMSE': max_,
                          'Average':avg,
                         'Min RMSE': min_})

plt.title('Parameter Optimisation')
plt.plot(evolution['Generation'], evolution['Min AUROC'], 'b', color = 'C1',
         label = 'Min')
plt.plot(evolution['Generation'], evolution['Average'], 'b', color = 'C2',
         label = 'Average')
plt.plot(evolution['Generation'], evolution['Max RMSE'], 'b', color = 'C3',
         label= 'Max')


plt.legend(loc = 'lower right')
plt.ylabel('RMSE')
plt.xlabel('Generation')
plt.xticks([0,5,10,15,20, 25, 30, 35, 40 ,45, 50])

In [ ]:
X_train, Y_train,_maxtrain, _mintrain, _meantrain, _stdtrain = preparing_data(df_train_micetreated, features= final_feature_list, lag = 24, normalize = 'minmax', category = '1ofN', onlyHour = , onlyMonth = )
X_test, Y_test,_maxtest, _mintest, _meantest, _stdtest = preparing_data(df_test_micetreated, features= final_feature_list, lag = 24, normalize = 'minmax', category = '1ofN', onlyHour = , onlyMonth = )

In [ ]:
regressor = SVR(kernel = svr_best_parameters_1[0], C=svr_best_parameters_1[1], epsilon=svr_best_parameters_1[2])
regressor.fit(X_train, Y_train)

testpredictSVR = multistep_prediction(regressor, X_test, 24)

orig_y_eval_test = Y_test*(_maxtest - _mintest) + _mintest
orig_y_hat_test_svr = testpredictSVR*(_maxtest - _mintest) + _mintest

RMSE_test_svr = (mean_squared_error(orig_y_eval_test, orig_y_hat_test_svr))**0.5
mape_svr = MAPE(orig_y_eval_test, orig_y_hat_test_svr)

print(f'RMSE: {RMSE_test_svr}')
print(f'MAPE: {mape_svr}')

In [ ]:
classification_real_day = mask_classification(orig_y_eval_test_day)
classification_forecasting_svr_day = mask_classification(orig_y_hat_test_svr_day)

print(confusion_matrix(classification_real_day, classification_forecasting_svr_day))
cm_svr_day = confusion_matrix(classification_real_day, classification_forecasting_svr_day)

cmn_day = cm_svr_day.astype('float') / cm_svr_day.sum(axis=1)[:, np.newaxis]
sns.heatmap(cmn_day,annot=True)
plt.ylabel('Actual')
plt.xlabel('Predicted')

**Previsão Horizonte de 1 dia**

**Seleção de hiper-parâmetros com algoritmos genéticos**

In [ ]:
from deap import base, creator, tools, algorithms
import random

def mutate(individual):
    
    gene = random.randint(0,7) #select which parameter to mutate
    if gene == 0:
        if individual[0] == 3:
            individual[0] = random.choice([6, 12, 24])
        elif individual[0] == 6:
            individual[0] = random.choice([3, 12, 24])
        elif individual[0] == 12:
            individual[0] = random.choice([3, 6, 24])
        else:
            individual[0] = random.choice([3, 6, 12])
    elif gene == 1:
        if individual[1] == 'minmax':
            individual[1] = 'standard'
        else:
            individual[1] = 'minmax'
    elif gene == 2:
        if individual[2] == 'numeric':
            individual[2] = random.choice(['1ofN', 'binary'])
        elif individual[2] == '1ofN':
            individual[2] = random.choice(['numeric', 'binary'])
        else:
            individual[2] = random.choice(['numeric', '1ofN'])
        
    elif gene == 3:
        if individual[3] == True:
            individual[3] = False
        else:
            individual[3] = True

    elif gene == 4:
        if individual[4] == True:
            individual[4] = False
        else:
            individual[4] = True
    if gene == 5:
        if individual[5] == 'linear':
            individual[5] = random.choice(['poly', 'rbf', 'sigmoid'])
        elif individual[5] == 'poly':
            individual[5] = random.choice(['linear', 'rbf', 'sigmoid'])
        elif individual[5] == 'rbf':
            individual[5] = random.choice(['linear', 'poly', 'sigmoid'])
        else:
            individual[5] = random.choice(['linear', 'poly', 'rbf'])
        
    elif gene == 6:
        individual[6] = random.uniform(lower_C, upper_C)
            
    elif gene == 7:
        individual[7] = random.uniform(lower_epsilon, upper_epsilon)
        
    return individual,


def evaluate(individual):
    '''
    build and test a model based on the parameters in an individual and return
    the AUROC value
    '''
    # extract the values of the parameters from the individual chromosome
    _lag = individual[0]
    _normalize = individual[1]
    _category = individual[2]
    _month = individual[3]
    _hour = individual[4]
    _kernel = individual[5]
    _C = individual[6]
    _epsilon = individual[7]
    
    # build the model
    print(f'Train_Lag: {_lag}')
    print(f'Train_Norm: {_normalize}')
    print(f'Train_Category: {_category}')
    print(f'Train_Hour: {_hour}')
    print(f'Train_Month: {_month}')
    X_train, Y_train,_maxtrain, _mintrain, _meantrain, _stdtrain = preparing_data(df_train_micetreated, features = final_feature_list, lag = _lag, normalize = _normalize, category = _category, onlyHour = _hour, onlyMonth = _month)
    print(f'Test_Lag: {_lag}')
    print(f'Test_Norm: {_normalize}')
    print(f'Test_Category: {_category}')
    print(f'Test_Hour: {_hour}')
    print(f'Test_Month: {_month}')
    X_test, Y_test,_maxtest, _mintest, _meantest, _stdtest = preparing_data(df_test_micetreated, features = final_feature_list,lag = _lag, normalize = _normalize, category = _category, onlyHour = _hour, onlyMonth = _month)
    
    
    print(f'kernel: {_kernel}')
    print(f'C: {_C}')
    print(f'epsilon: {_epsilon}')
    
    regressor = SVR(kernel=_kernel, 
                    C=_C, 
                    epsilon=_epsilon)
    regressor.fit(X_train, Y_train)
    
    testpredictSVR = multistep_prediction(regressor, X_test[-168:], _lag)

    orig_y_eval_test = Y_test[-168:]*(_maxtest - _mintest) + _mintest
    orig_y_hat_test_svr = testpredictSVR*(_maxtest - _mintest) + _mintest

    RMSE_test_svr = (mean_squared_error(orig_y_eval_test, orig_y_hat_test_svr))**0.5
    

    print(f'RMSE: {RMSE_test_svr}')
    print('------'*20)
    return RMSE_test_svr,


creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # Maximise the fitness function value
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Possible parameter values
lag_option = [3, 6, 12, 24]
norm = ['minmax', 'standard']
cat = ['numeric', '1ofN', 'binary']
hour = [True, False]
month = [True, False]
list_kernel = ['linear', 'poly', 'rbf', 'sigmoid']
lower_C, upper_C = 0.1, 50
lower_epsilon, upper_epsilon = 0.001,1 

N_CYCLES = 1
toolbox.register("attr_lag", random.choice, lag_option)
toolbox.register("attr_normalization", random.choice, norm)
toolbox.register("attr_category", random.choice, cat)
toolbox.register("attr_hour", random.choice, hour)
toolbox.register("attr_month", random.choice, month)
toolbox.register("attr_kernel", random.choice, list_kernel)
toolbox.register("attr_C", random.uniform, lower_C, upper_C)
toolbox.register("attr_epochs", random.uniform, lower_epsilon, upper_epsilon)

toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_lag,toolbox.attr_normalization, toolbox.attr_category,
                  toolbox.attr_hour,toolbox.attr_month,toolbox.attr_kernel,toolbox.attr_C, 
                  toolbox.attr_epochs), n=N_CYCLES)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate",mutate)
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("evaluate", evaluate)

population_size = 4
crossover_probability = 0.75
mutation_probability = 0.05
number_of_generations = 50

pop = toolbox.population(n=population_size)
hof = tools.HallOfFame(3)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=crossover_probability, stats = stats, 
                               mutpb = mutation_probability, ngen=number_of_generations, halloffame=hof, 
                               verbose=True) 

svr_best_parameters_1 = hof[0] # save the optimal set of parameters
svr_best_parameters_2 = hof[1]
svr_best_parameters_3 = hof[2]
print(svr_best_parameters_1)
print(svr_best_parameters_2)
print(svr_best_parameters_3)

gen = log.select("gen")
max_ = log.select("max")
avg = log.select("avg")
min_ = log.select("min")

In [ ]:
evolution = pd.DataFrame({'Generation': gen,
                         'Max RMSE': max_,
                          'Average':avg,
                         'Min RMSE': min_})

plt.title('Parameter Optimisation')
plt.plot(evolution['Generation'], evolution['Min AUROC'], 'b', color = 'C1',
         label = 'Min')
plt.plot(evolution['Generation'], evolution['Average'], 'b', color = 'C2',
         label = 'Average')
plt.plot(evolution['Generation'], evolution['Max RMSE'], 'b', color = 'C3',
         label= 'Max')


plt.legend(loc = 'lower right')
plt.ylabel('RMSE')
plt.xlabel('Generation')
plt.xticks([0,5,10,15,20, 25, 30, 35, 40 ,45, 50])

In [ ]:
X_train, Y_train,_maxtrain, _mintrain, _meantrain, _stdtrain = preparing_data(df_train_micetreated, features= final_feature_list, lag = 24, normalize = 'minmax', category = '1ofN', onlyHour = , onlyMonth = )
X_test, Y_test,_maxtest, _mintest, _meantest, _stdtest = preparing_data(df_test_micetreated, features= final_feature_list, lag = 24, normalize = 'minmax', category = '1ofN', onlyHour = , onlyMonth = )

In [ ]:
regressor = SVR(kernel = svr_best_parameters_1[0], C=svr_best_parameters_1[1], epsilon=svr_best_parameters_1[2])
regressor.fit(X_train, Y_train)

testpredictSVR = multistep_prediction(regressor, X_test, 24)

orig_y_eval_test = Y_test*(_maxtest - _mintest) + _mintest
orig_y_hat_test_svr = testpredictSVR*(_maxtest - _mintest) + _mintest

RMSE_test_svr = (mean_squared_error(orig_y_eval_test, orig_y_hat_test_svr))**0.5
mape_svr = MAPE(orig_y_eval_test, orig_y_hat_test_svr)

print(f'RMSE: {RMSE_test_svr}')
print(f'MAPE: {mape_svr}')

In [ ]:
classification_real_day = mask_classification(orig_y_eval_test_day)
classification_forecasting_svr_day = mask_classification(orig_y_hat_test_svr_day)

print(confusion_matrix(classification_real_day, classification_forecasting_svr_day))
cm_svr_day = confusion_matrix(classification_real_day, classification_forecasting_svr_day)

cmn_day = cm_svr_day.astype('float') / cm_svr_day.sum(axis=1)[:, np.newaxis]
sns.heatmap(cmn_day,annot=True)
plt.ylabel('Actual')
plt.xlabel('Predicted')